# CourtPressGER: German Court Press Release Dataset

- Ein Datensatz bestehend aus :
-   6,5k deutscher höchstgerichtlicher Urteile und zugehöriger Pressemitteilungen und
-   zugehörige synthetische Prompts, um aus den Urteilen Pressemitteilungen zu generieren
- Baselines

In [ ]:
# GPU-Beschleunigung Setup
try:
    import cudf
    import cuml
    import cupy as cp
    from cuml.metrics import pairwise_distances
    from cuml.preprocessing import normalize
    USE_GPU = True
    print("GPU-Beschleunigung ist aktiviert (RAPIDS-Bibliotheken geladen)")
except ImportError:
    USE_GPU = False
    print("GPU-Beschleunigung ist nicht verfügbar, verwende CPU-Version")

# Standard Data Science Imports
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Transformer & NLP
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import spacy

# Visualisierung
import matplotlib.pyplot as plt
import seaborn as sns

# Setze Warnungen und Seed
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)
if USE_GPU:
    cp.random.seed(42)

# Lade die Daten
try:
    if USE_GPU:
        df = cudf.read_csv('data/german_courts.csv')
        print("Daten mit GPU-Beschleunigung geladen")
    else:
        df = pd.read_csv('data/german_courts.csv')
        print("Daten mit CPU geladen")
    print(f"Datensatzgröße: {len(df)} Einträge")
except Exception as e:
    print(f"Fehler beim Laden der Daten: {e}")

## Model Setup und Training

In diesem Abschnitt laden und trainieren wir die Modelle für die Verarbeitung der Gerichtsurteile und Pressemitteilungen.
Wir nutzen die GPU-Beschleunigung wo möglich, mit Fallback auf CPU-Implementierungen.

In [ ]:
# Lade bereinigte Daten
try:
    if USE_GPU:
        clean_df = cudf.read_csv('cleaned_data/cleaned_combined_methods.csv')
    else:
        clean_df = pd.read_csv('cleaned_data/cleaned_combined_methods.csv')
    print(f"Bereinigte Daten geladen: {len(clean_df)} Einträge")
except Exception as e:
    print(f"Fehler beim Laden der bereinigten Daten: {e}")

# Vorbereitung der Modelle
def prepare_model_inputs(texts, tokenizer, max_length=512):
    """Bereitet Texte für die Modellverarbeitung vor"""
    # Tokenisierung mit Padding und Truncation
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
    if USE_GPU:
        return {k: v.cuda() for k, v in inputs.items()}
    return inputs

# Lade Modell und Tokenizer
model_name = 'deepset/gbert-large'  # Deutsches BERT Modell
print(f"Lade {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

if USE_GPU:
    model = model.cuda()
    print("Modell auf GPU geladen")
else:
    print("Modell auf CPU geladen")

# Sentence Transformer für Ähnlichkeitsvergleiche
st_model = SentenceTransformer('T-Systems-onsite/german-roberta-sentence-transformer-v2')
if USE_GPU:
    st_model = st_model.cuda()

# Beispiel-Inference
sample_size = min(5, len(clean_df))
sample_texts = clean_df['summary'].head(sample_size).to_pandas() if USE_GPU else clean_df['summary'].head(sample_size)

print("\nFühre Beispiel-Inference durch...")
with torch.no_grad():
    inputs = prepare_model_inputs(sample_texts.tolist(), tokenizer)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token embeddings

print(f"Embedding-Dimensionen: {embeddings.shape}")

if USE_GPU:
    # Berechne Ähnlichkeitsmatrix mit cupy
    embeddings_gpu = cp.array(embeddings.cpu().numpy())
    similarity_matrix = cp.matmul(embeddings_gpu, embeddings_gpu.T)
    print("\nÄhnlichkeitsmatrix (GPU-beschleunigt):")
    print(cp.asnumpy(similarity_matrix))
else:
    # Berechne Ähnlichkeitsmatrix auf CPU
    similarity_matrix = np.matmul(embeddings.numpy(), embeddings.numpy().T)
    print("\nÄhnlichkeitsmatrix (CPU):")
    print(similarity_matrix)

## Training und Evaluation

In diesem Abschnitt trainieren wir das Modell auf den Trainingsdaten und evaluieren die Performance.
Die GPU-Beschleunigung wird für Batch-Verarbeitung und Matrixoperationen genutzt.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# Batch-Größe basierend auf verfügbarem GPU-Speicher anpassen
BATCH_SIZE = 32 if USE_GPU else 16

def create_dataloader(texts, labels, tokenizer):
    """Erstellt einen DataLoader für das Training"""
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    if USE_GPU:
        input_ids = inputs['input_ids'].cuda()
        attention_mask = inputs['attention_mask'].cuda()
        labels = torch.tensor(labels).cuda()
    else:
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        labels = torch.tensor(labels)
    
    dataset = TensorDataset(input_ids, attention_mask, labels)
    return DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Erstelle Train/Val Split
train_texts = clean_df['judgement'].sample(frac=0.8, random_state=42)
val_texts = clean_df['judgement'].drop(train_texts.index)
train_labels = clean_df['summary'][train_texts.index]
val_labels = clean_df['summary'][val_texts.index]

# Konvertiere zu Listen für die Tokenisierung
if USE_GPU:
    train_texts = train_texts.to_pandas().tolist()
    val_texts = val_texts.to_pandas().tolist()
    train_labels = train_labels.to_pandas().tolist()
    val_labels = val_labels.to_pandas().tolist()
else:
    train_texts = train_texts.tolist()
    val_texts = val_texts.tolist()
    train_labels = train_labels.tolist()
    val_labels = val_labels.tolist()

# Erstelle DataLoader
train_dataloader = create_dataloader(train_texts, train_labels, tokenizer)
val_dataloader = create_dataloader(val_texts, val_labels, tokenizer)

# Training Loop mit GPU-Beschleunigung
def train_epoch(model, dataloader, optimizer, scheduler=None):
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc='Training')
    
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()
            
        progress_bar.set_postfix({'loss': loss.item()})
    
    return total_loss / len(dataloader)

# Evaluation
@torch.no_grad()
def evaluate(model, dataloader):
    model.eval()
    total_loss = 0
    all_predictions = []
    all_labels = []
    
    for batch in tqdm(dataloader, desc='Evaluating'):
        input_ids, attention_mask, labels = batch
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        total_loss += loss.item()
        
        predictions = outputs.logits.argmax(dim=-1)
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
    return (
        total_loss / len(dataloader),
        classification_report(all_labels, all_predictions)
        confusion_matrix(all_labels, all_predictions)
    )

# Speichern der Modelle
def save_model(model, epoch, optimizer, loss, path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, path)

# Haupttrainingsschleife
print("Starte Training...")
NUM_EPOCHS = 3
best_val_loss = float('inf')

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoche {epoch+1}/{NUM_EPOCHS}")
    
    # Training
    train_loss = train_epoch(model, train_dataloader, optimizer)
    print(f"Training Loss: {train_loss:.4f}")
    
    # Evaluation
    val_loss, val_report, val_cm = evaluate(model, val_dataloader)
    print(f"Validation Loss: {val_loss:.4f}")
    print("\nKlassifikationsbericht:")
    print(val_report)
    
    # Speichere bestes Modell
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        save_model(model, epoch, optimizer, val_loss, 'best_model.pt')
        print("Neues bestes Modell gespeichert!")

print("Training abgeschlossen!")